<a href="https://colab.research.google.com/github/LWUGDEV/WOFFSample/blob/main/LINE_WORKS_%E3%83%A1%E3%83%83%E3%82%BB%E3%83%BC%E3%82%B8%E9%80%81%E5%8F%97%E4%BF%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Service Account 認証によるトークン生成

1. Private Key のアップロード

In [ ]:
from google.colab import files

uploaded = files.upload()
PRIVATE_KEY_PATH = list(uploaded.keys())[0]
with open(PRIVATE_KEY_PATH, 'r') as file:
    PRIVATE_KEY = file.read()

Saving private_20230910070644.key to private_20230910070644 (1).key


2. 設定部分

In [ ]:
CLIENT_ID = "i2mhVFCy4l9ovJqWYhUY"
CLIENT_SECRET = "CpOcZAtbEU"
SCOPE = "bot"
SERVICE_ACCOUNT = "xjaht.serviceaccount@lwugdev"


3. JWTの設定、生成、トークン取得

In [ ]:
import jwt
import time
import json
import requests
from datetime import datetime, timedelta

# JWTのHeaderとPayloadを設定
header = {
    "alg": "RS256",
    "typ": "JWT"
}

# 現在のUTC時刻を取得して、iatとexpを設定
current_time = datetime.utcnow()
iat = int(time.mktime(current_time.timetuple()))
exp = int(time.mktime((current_time + timedelta(hours=1)).timetuple()))

payload = {
    "iss": CLIENT_ID,
    "sub": SERVICE_ACCOUNT,
    "iat": iat,
    "exp": exp
}

# JWTの生成
assertion = jwt.encode(payload, PRIVATE_KEY, algorithm="RS256", headers=header)

# POSTリクエストのためのデータを準備
data = {
    'assertion': assertion,
    'grant_type': "urn:ietf:params:oauth:grant-type:jwt-bearer",
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
    'scope': SCOPE
}

headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

# API エンドポイントに POST リクエストを送信してトークンを取得
response = requests.post('https://auth.worksmobile.com/oauth2/v2.0/token', data=data, headers=headers)

# 応答の確認
if response.status_code == 200:
    token_info = response.json()
    access_token = token_info.get('access_token')
    print(f"Access Token: {access_token}")
else:
    print(f"Failed to retrieve token. Status code: {response.status_code}, Response: {response.text}")

Access Token: jp1AAABHSrSkhtD95XnoGIDswHbHkguZsDAE+zI0JAgdSZjyBstlwHnI8DpxDMfikEfWElU2WE1bLyhc/ouXAJB0HBfRSvL2eLZxMI4lVDWdIrNICwD9q7a0lDOX7Ck+q8vR1Rv1ldr4M8ASR5O2tFNBEA2/z7FIKuAw6OuTFnh6xpdi0W/QdqOTHzRjhJWvXkEkvrC9iX94Slr3DM83EOxDOGBV6SDvb7rXT7MX96knkzh+s49TvRlvU+IG8y5qEKOl3IB2FnNz/wjM2y+FUQuW2zAebXLwvCqpuDahQa5HRx+IMrT+YHzwAz3oqa1mrOVEehCbWvLO3gI1cwR8xRwwWtL7/XLYqC/8ePUcgG6wQFC25ryo8JMhaIWgKPAdQH8bKcV5Q==.kwiu9yNovfcs8Rumz2QSOg


# LINE WORKS の Bot からメッセージを送信

メッセージ送信

In [ ]:
import json
import requests

# 上のセルで取得したトークン
TOKEN = token_info.get('access_token')

# 設定情報
BOT_ID = '6264248'  # 例: '2000001'
USER_ID = 'iwaoh@lwugdev'  # 例: 'userf7da-f82c-4284-13e7-030f3b4c756x'

API_URL = f'https://www.worksapis.com/v1.0/bots/{BOT_ID}/users/{USER_ID}/messages'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {TOKEN}'
}

# メッセージの内容を定義
data = {
    "content": {
        "type": "text",
        "text": "Hello from LINE Works bot!"
    }
}

response = requests.post(API_URL, headers=headers, data=json.dumps(data))

# 応答の確認
if response.status_code == 201:
    print("Message sent successfully!")
else:
    print(f"Failed to send message. Status code: {response.status_code}, Response: {response.text}")


Message sent successfully!


# LINE WORKS の Bot でメッセージ受信

In [ ]:
!pip uninstall flask-ngrok -y

Found existing installation: flask-ngrok 0.0.25
Uninstalling flask-ngrok-0.0.25:
  Successfully uninstalled flask-ngrok-0.0.25


In [ ]:
!pip install flask-ngrok

  Using cached flask_ngrok-0.0.25-py3-none-any.whl (3.1 kB)


In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import threading

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/lineworks', methods=['POST'])
def receive_message():
    print("Received a request!")  # リクエスト受信をログに記録
    data = request.json
    print(data)  # 受信データをログに記録
    if data['type'] == 'message':
        user_id = data['source']['userId']
        text = data['content']['text']
        print(f"Message from {user_id}: {text}")  # メッセージ内容をログに記録
        return jsonify(status="success"), 200
    return jsonify(status="not a message"), 400



def run_on_another_thread():
    app.run()

threading.Thread(target=run_on_another_thread).start()

import time
time.sleep(1)

# Get all ngrok tunnels
tunnels = requests.get("http://localhost:4040/api/tunnels").json()['tunnels']

# Extract the HTTPS URL
ngrok_https_url = next((tunnel['public_url'] for tunnel in tunnels if tunnel['proto'] == 'https'), None)

if not ngrok_https_url:
    raise ValueError("No HTTPS URL found from ngrok. Make sure ngrok is running properly.")

# Use the `access_token` from the previous cell
headers = {
    'Authorization': f"Bearer {access_token}",
    'Content-Type': 'application/json'
}

data = {
  "callbackUrl": f"{ngrok_https_url}/lineworks"
}

response = requests.patch('https://www.worksapis.com/v1.0/bots/6264248', headers=headers, json=data)

if response.status_code == 200:
    print("Bot callbackUrl updated successfully!")
else:
    print(f"Failed to update bot callbackUrl. Status code: {response.status_code}, Response: {response.text}")


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


 * Running on http://2131-34-138-36-55.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
Bot callbackUrl updated successfully!


In [ ]:
import os

# ngrokのAUTH TOKENを設定
os.environ['NGROK_AUTH_TOKEN'] = '2V96WfmNLN0x5WPmDcHjwXjCAqE_6BdiUnv7ctXdPxe2zvd1T'

# ngrokをインストール
!pip install pyngrok

from pyngrok import ngrok


In [ ]:
ngrok config add-authtoken 2V96WfmNLN0x5WPmDcHjwXjCAqE_6BdiUnv7ctXdPxe2zvd1T

SyntaxError: ignored